In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import time, datetime
warnings.filterwarnings('ignore')

### 아래 가이드에 따라서 고객 구매데이터를 생성하고 아래사항을 수행하세요(관측치 10000개)

#### Q1. 고객 속성(id, 성별, 연령, 거주지역) 데이터와 거래 데이터를 통합한 데이터 프레임 df를 생성하세요.

거래 번호
- id : 1 ~ 10000번 일련번호 - 개시 후 1만 건의 거래가 있었고 걍 거기에 넘버링한 거라고 이해해보자

고객별 속성 데이터
- cusno : 고객번호 1000개 랜덤
- gender : 0,1 정수 난수 생성
- age : 0 ~ 80사이 정수 난수 생성
- region : 1 ~ 10 사이 정수 난수 생성

거래 데이터
- day : 요일
- date : 2020-01-01 ~ 2021-12-31 사이의 날짜 1000개 생성
- product_20 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10,20,30
- product_21 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10,20,30
- price_20 : 1000 ~ 50000 사이 실수 난수 생성
- price_21 : 1000 ~ 50000 사이 실수 난수 생성
- quantity_20 : 1 ~ 100 사이 정수 난수 생성
- quantity_21 : 1 ~ 100 사이 정수 난수 생성
- time_20 : 01 ~ 24 사이 정수 난수 생성
- time_21 : 01 ~ 24 사이 정수 난수 생성
- amount_20 : price_20 * quantity_20
- amount_21 : price_21 * quantity_21
- sales : 20년 대비 21년구매금액이 증가면 1, 감소면 0

#### Q2. df를 수정하여 5가지 이상의 인사이트를 포함한 데이터 셋으로 변환하세요

#### Q3. df 를 탐색적 분석을 통하여 인사이트를 도출하세요.


In [3]:
# 뭐 매출이 감소했는데 고가제품 때문이었더라~
# 어느 시즌에 매출이 몰렸다, 이 기회를 잘 활용하자~ 등등

- day : 요일
- date : 2020-01-01 ~ 2021-12-31 사이의 날짜


- 두 해의 날짜를 연별로 랜덤하게 생성하는 걸 만들어서 거래 년도에 해당하는 날짜를 랜덤하게 적용
- product_20 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10,20,30
- product_21 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10,20,30
- price_20 : 1000 ~ 50000 사이 실수 난수 생성
- price_21 : 1000 ~ 50000 사이 실수 난수 생성
- quantity_20 : 1 ~ 100 사이 정수 난수 생성
- quantity_21 : 1 ~ 100 사이 정수 난수 생성
- time_20 : 01 ~ 24 사이 정수 난수 생성
- time_21 : 01 ~ 24 사이 정수 난수 생성
- amount_20 : price_20 * quantity_20
- amount_21 : price_21 * quantity_21
- sales : 20년 대비 21년구매금액이 증가면 1, 감소면 0

In [3]:
ID = pd.Series(np.arange(1,10001),name='거래번호')

# 고객 데이터프레임 customer
#np.random.seed(0)
cusno = pd.Series(np.arange(1,1001),name='cusno')
gender = pd.Series(np.random.randint(0,2,1000),name='gender')
age = pd.Series(np.random.randint(1,81,1000),name='age')
region = pd.Series(np.random.randint(1,11,1000),name='region')
customer = pd.concat((cusno,gender,age,region),axis=1)

# 거래 데이터프레임 data
# 구매일/시간/구매가격/구매물품코드/상품구분/구매량
df = customer.sample(n=10000,replace=True).set_index(ID)
# 시간 관련
start = pd.to_datetime('2020-01-01')
end = pd.to_datetime('2021-12-31')
date = pd.date_range(start,end,freq="D")
date2 = date.to_frame(index=False) # 인덱스 떼고 데이터프레임화
df['date'] = date2.sample(10000,replace=True).set_index(ID)
df['timedata'] = pd.to_datetime(df.date)
# date는 단순 문자 시리즈였지면 이 작업을 해주면 DateTime index로 써 먹을 수 있다!
df['Year'] = df.timedata.dt.year
df['Month'] = df.timedata.dt.month
df['Day'] = df.timedata.dt.day
df['time'] = pd.Series(np.random.randint(1,25,10001))
df.drop('date',axis=1,inplace=True)

# 요일 관련
def days(y,m,d):
    dli=['MON','TUE','WED','THU','FRI','SAT','SUN']
    return dli[datetime.date(y,m,d).weekday()]
df['Day'] = df.apply(lambda x:days(x['Year'],x['Month'],x['Day']),axis=1)

# 거래 관련
df['price'] = pd.Series(np.random.randint(10,500,10001)*100,name='price')
df['product'] = pd.cut(df['price'],
                bins=3,
                labels = [30,20,10])
df['quantity'] = pd.Series(np.random.randint(1,101,10001))
df['총 구매액'] = df.price*df.quantity
display(df)

,cusno,gender,age,region,timedata,Year,Month,Day,time,price,product,quantity,총 구매액
거래번호,,,,,,,,,,,,,
1,563,0,60,5,2020-11-04,2020,11,WED,22,42000,10,43,1806000
2,986,1,45,1,2020-11-11,2020,11,WED,17,46500,10,48,2232000
3,659,1,36,1,2021-03-07,2021,3,SUN,4,1300,30,22,28600
4,459,0,72,7,2020-04-19,2020,4,SUN,1,45900,10,87,3993300
5,785,1,71,4,2020-11-09,2020,11,MON,14,42700,10,94,4013800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,175,1,71,4,2021-04-04,2021,4,SUN,5,44900,10,62,2783800
9997,9,1,28,1,2020-06-22,2020,6,MON,7,26700,20,88,2349600
9998,248,0,14,5,2021-02-10,2021,2,WED,12,21300,20,77,1640100


두 데이터프레임을 합칠 때에<br>
고객 데이터에서 랜덤 행을 가져와서 1만개의 관측치를 채우고<br>
고객넘버/성별/나이/지역/구매일/시간/구매물품코드/구매가격/구매량 으로 묶는 게 보기 좋을 듯<br>
구매물품코드는 빈을 써서 3등분하고 라벨을 10,20,30으로 붙이자

In [4]:
# df에서 연도별로 나눠보자.
df1 = df.copy()
display(df1[df1.Year==2020].head(),df1[df1.Year==2021].head())

# 2020년 총 정리
print(f"2020년 총 판매 액수: {df1.loc[df1.Year==2020,'총 구매액'].sum()}원")
print(f"2020년 총 판매 건수: {len(df1.loc[df1.Year==2020])}건")
# 2021년 총 정리
print(f"2021년 총 판매 액수: {df1.loc[df1.Year==2021,'총 구매액'].sum()}원")
print(f"2020년 총 판매 건수: {len(df1.loc[df1.Year==2021])}건")

,cusno,gender,age,region,timedata,Year,Month,Day,time,price,product,quantity,총 구매액
거래번호,,,,,,,,,,,,,
1,563,0,60,5,2020-11-04,2020,11,WED,22,42000,10,43,1806000
2,986,1,45,1,2020-11-11,2020,11,WED,17,46500,10,48,2232000
4,459,0,72,7,2020-04-19,2020,4,SUN,1,45900,10,87,3993300
5,785,1,71,4,2020-11-09,2020,11,MON,14,42700,10,94,4013800
8,805,1,42,7,2020-08-18,2020,8,TUE,18,12700,30,18,228600


,cusno,gender,age,region,timedata,Year,Month,Day,time,price,product,quantity,총 구매액
거래번호,,,,,,,,,,,,,
3,659,1,36,1,2021-03-07,2021,3,SUN,4,1300,30,22,28600
6,905,1,79,5,2021-11-01,2021,11,MON,4,35800,10,95,3401000
7,7,0,20,7,2021-11-04,2021,11,THU,12,34900,10,23,802700
9,182,0,52,10,2021-10-04,2021,10,MON,19,33300,20,17,566100
10,229,0,72,4,2021-04-24,2021,4,SAT,23,22100,20,100,2210000


2020년 총 판매 액수: 6479885200원
2020년 총 판매 건수: 5032건
2021년 총 판매 액수: 6471033200원
2020년 총 판매 건수: 4968건


In [5]:
# 2020년 중 가장 판매액이 높았던 달을 찾아볼까?
df2020 = df1[df1.Year==2020]
display(df2020.head(3))
bestm = []
for i in range(1,13):
    bestm.append(df2020.loc[df2020.Month==i,'총 구매액'].sum())
bestm2 = np.array(bestm)
print(f'{bestm.index(bestm2.max())+1}월, 총 판매액 {bestm2.max()}원')

,cusno,gender,age,region,timedata,Year,Month,Day,time,price,product,quantity,총 구매액
거래번호,,,,,,,,,,,,,
1,563,0,60,5,2020-11-04,2020,11,WED,22,42000,10,43,1806000
2,986,1,45,1,2020-11-11,2020,11,WED,17,46500,10,48,2232000
4,459,0,72,7,2020-04-19,2020,4,SUN,1,45900,10,87,3993300


3월, 총 판매액 631512200원


In [6]:
# 고객 넘버 16번은 2년 동안 얼마나 구매했을까?
print(f"{len(df[df.cusno==16])}회 구매, {df.loc[df.cusno==16,'총 구매액'].sum()}원")
df[df.cusno==16]

12회 구매, 16063400원


,cusno,gender,age,region,timedata,Year,Month,Day,time,price,product,quantity,총 구매액
거래번호,,,,,,,,,,,,,
194,16,0,58,8,2020-01-12,2020,1,SUN,18,24800,20,18,446400
1306,16,0,58,8,2020-05-31,2020,5,SUN,18,23600,20,24,566400
1401,16,0,58,8,2020-12-18,2020,12,FRI,14,44100,10,84,3704400
1886,16,0,58,8,2021-01-03,2021,1,SUN,13,21600,20,8,172800
2613,16,0,58,8,2021-07-30,2021,7,FRI,1,41700,10,89,3711300
5710,16,0,58,8,2020-03-14,2020,3,SAT,24,1800,30,55,99000
7185,16,0,58,8,2020-03-24,2020,3,TUE,17,1100,30,90,99000
7303,16,0,58,8,2021-02-03,2021,2,WED,18,5300,30,95,503500
7999,16,0,58,8,2021-02-05,2021,2,FRI,14,15700,30,70,1099000


In [7]:
# 2021년 중 가장 판매액이 높았던 달을 찾아볼까?
df2021 = df1[df1.Year==2021]
display(df2021.head(3))
bestm = []
for i in range(1,13):
    bestm.append(df2021.loc[df2021.Month==i,'총 구매액'].sum())
bestm2 = np.array(bestm)
print(f'{bestm.index(bestm2.max())+1}월, 총 판매액 {bestm2.max()}원')

,cusno,gender,age,region,timedata,Year,Month,Day,time,price,product,quantity,총 구매액
거래번호,,,,,,,,,,,,,
3,659,1,36,1,2021-03-07,2021,3,SUN,4,1300,30,22,28600
6,905,1,79,5,2021-11-01,2021,11,MON,4,35800,10,95,3401000
7,7,0,20,7,2021-11-04,2021,11,THU,12,34900,10,23,802700


5월, 총 판매액 578474400원


In [8]:
# 매해 어느 시간대에 가장 구매 건 수가 많았을까?
bestt = []
for i in range(1,25):
    bestt.append(len(df2020.loc[df2020.time==i]))
bestt2 = np.array(bestt)
print(f'{bestt.index(bestt2.max())+1}시, 총 거래 건수 {bestt2.max()}건')

bestt = []
for i in range(1,25):
    bestt.append(len(df2021.loc[df2021.time==i]))
bestt2 = np.array(bestt)
print(f'{bestt.index(bestt2.max())+1}시, 총 거래 건수 {bestt2.max()}건')

6시, 총 거래 건수 233건
5시, 총 거래 건수 231건


In [9]:
# 그루핑을 수행해 보자
# 연령대의 범주화 선행

def agecat(age):
    cat = ''
    if age<10: cat='아동'
    elif age<20: cat='10대'
    elif age<=30: cat='20대'
    elif age<=40: cat='30대'
    elif age<=50: cat='40대'
    elif age<=60: cat='50대'
    elif age<=60: cat='60대'
    else: cat='노년층'
    return cat
df20 = df2020.copy()
df20.drop(['cusno','region'],axis=1,inplace=True)
df20['age'] = df20.age.apply(lambda x:agecat(x))
df20
gr20 = df20.groupby(['gender','age','product'])
gr20.mean().head()
# 랜덤 수를 뽑다보니 아무래도 평균 값은 중앙에 가깝게 나온다 ㅋㅋ

Year     Month       time         price   quantity  \
gender age product                                                         
0      10대 30       2020.0  6.213483  12.584270   8965.168539  54.955056   
           20       2020.0  6.637795  12.543307  24704.724409  49.362205   
           10       2020.0  6.314607  12.730337  42024.719101  54.337079   
       20대 30       2020.0  6.118280  13.440860   9143.010753  45.634409   
           20       2020.0  6.109244  12.966387  25461.344538  46.663866   

                           총 구매액  
gender age product                
0      10대 30       4.733719e+05  
           20       1.201211e+06  
           10       2.285444e+06  
       20대 30       4.199366e+05  
           20       1.198327e+06

코로나 때문에 구매 시간의 변동이 생겼다.<br>
라고 했을 때 구매 시간의 비중을 늘리는 게 아니라<br>
해당 구매 시간에 있었던 컬럼을 쫙 뽑아! 그 다음에 거기 구매갯수 늘려!<br>
함수 자체의 확률을 바꾸라는 게 아니라 내가 뽑고 싶은 구간의 데이터를 뽑은 다음<br>
내 입맛에 맞게 결과값만 바꾸라는 뜻임

#### 1. 코시국 에디션
    - 영업 제한: 8월~11월 평일 오후 9시(21시) 이후 구매 수량 60% 축소
    - 보복 소비: 12월 고가품목(10) 구매수량 150% 증대
    - 재난지원금: 9월 중/저가품목(20,30) 구매수량 300% 증대
#### 2. 옳게 된 소비
    - 주말 특수: 주말 오후 4~7시 전체 구매수량 기존값의 140%로 바꾸기
    - 경제 권력층: 전체 30대~50대 구매수량 170% 증대, 10대 전체 구매수량 70% 축소
#### 3. 시즈널 이펙트
    - 민족대명절: 2월, 9월 30세 이상 여성 고가품목 구매 수량 300% 증대
    - 곧 휴가철입니다: 7,8월 20대~40대 전체 구매수량 150% 증대
    - 신학기 경험치 2배: 3월 10대 중/고가품목 구매수량 140% 증대
#### 4. 인생의 쓴맛
    - I HATE MONDAYS: 전체 월요일 구매수량 50% 축소
    - 미미은행 발 위조지폐: 아동 범주 대상 전체 구매수량 5%로 축소
    - 못 살 이유는 딱히 없지만: 10대 대상 전체 고가품목 구매 수량 50% 축소
    - 심야영업 안 해요: 오전 9시 이전, 오후 11시 이후 구매수량 50% 축소
#### 5. 그냥 넣고 싶었어
    - 4월은 내 생일: 4월 16일 30대 여성(1) 고가품목(10) 구매수량 400% 증대


In [10]:
# 작업을 위한 전처리
# 만들어둔 데이트타임 인덱스를 인덱스로 붙이고 날짜순으로 정렬

mydf = df.copy()
mydf.set_index(mydf.timedata,inplace=True)
mydf.sort_index(ascending=True, inplace=True)
mydf

,cusno,gender,age,region,timedata,Year,Month,Day,time,price,product,quantity,총 구매액
timedata,,,,,,,,,,,,,
2020-01-01,578,0,64,3,2020-01-01,2020,1,WED,10,26500,20,2,53000
2020-01-01,862,0,4,4,2020-01-01,2020,1,WED,20,1600,30,64,102400
2020-01-01,97,0,27,4,2020-01-01,2020,1,WED,15,7100,30,79,560900
2020-01-01,525,0,77,4,2020-01-01,2020,1,WED,16,13700,30,23,315100
2020-01-01,149,1,34,3,2020-01-01,2020,1,WED,1,42200,10,1,42200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,826,1,72,8,2021-12-31,2021,12,FRI,4,47900,10,28,1341200
2021-12-31,883,1,30,4,2021-12-31,2021,12,FRI,8,16300,30,42,684600
2021-12-31,283,0,29,9,2021-12-31,2021,12,FRI,13,39900,10,2,79800


In [11]:
# 1. 코시국 에디션
# - 영업 제한: 8월~11월 평일 오후 9시(21시) 이후 구매 수량 60% 축소
# - 보복 소비: 12월 고가품목(10) 구매수량 150% 증대
# - 재난지원금: 9월 중/저가품목(20,30) 구매수량 300% 증대
mydf.loc[(mydf.Month>=8)&(mydf.Month<=11)&(mydf.time>=21),'quantity'] *=0.6 #(적용 ㅇㅋ)
mydf.loc[(mydf.Month==12)&(mydf['product']==10),'quantity']*=1.5 #(적용 ㅇㅋ)
mydf.loc[(mydf.Month==9)&(mydf['product']==30)|(mydf['product']==20),'quantity']*=3 #(적용 ㅇㅋ)

# 2. 옳게 된 소비
# - 주말 특수: 주말 오후 4~7시 전체 구매수량 기존값의 140%로 바꾸기
# - 경제 권력층: 전체 30대~50대 구매수량 170% 증대, 10대 전체 구매수량 70% 축소
mydf.loc[((mydf.Day=='SAT')|(mydf.Day=='SUN'))&(mydf.time>=16)&(mydf.time<=19),'quantity']*=1.4
mydf.loc[(mydf.age>=30)&(mydf.age<60),'quantity']*=1.7
mydf.loc[(mydf.age>=10)&(mydf.age<20),'quantity']*=0.7

# 3. 시즈널 이펙트
# - 민족대명절: 2월, 9월 30세 이상 여성 고객 고가품목 구매 수량 300% 증대
# - 곧 휴가철입니다: 7,8월 20대~40대 전체 구매수량 150% 증대
# - 신학기 경험치 2배: 3월 10대 중/고가품목 구매수량 140% 증대
mydf.loc[((mydf.Month==2)|(mydf.Month==9))&(mydf.age>=30)&(mydf.gender==1),'quantity']*=3
mydf.loc[((mydf.Month==7)|(mydf.Month==8))&(mydf.age>=20)&(mydf.age<50),'quantity']*=1.5
mydf.loc[((mydf['product']==10)|(mydf['product']==20))
         &(mydf.Month==3)&(mydf.age>=10)&(mydf.age<20),'quantity']*=1.4

# 4. 인생의 쓴맛
# - I HATE MONDAYS: 전체 월요일 구매수량 50% 축소
# - 미미은행 발 위조지폐: 아동 범주 대상 전체 구매수량 5%로 축소
# - 못 살 이유는 딱히 없지만: 10대 대상 전체 고가품목 구매 수량 50% 축소
# - 심야영업 안 해요: 오전 9시 이전, 오후 11시 이후 구매수량 50% 축소
mydf.loc[(mydf.Day=='MON'),'quantity']*=0.5
mydf.loc[(mydf.age<10),'quantity']*=0.05
mydf.loc[(mydf['product']==10)&(mydf.age>=10)&(mydf.age<20),'quantity']*=0.5
mydf.loc[(mydf.time<=9)|(mydf.time>=23),'quantity'] *=0.5

# 5. 그냥 넣고 싶었어
# - 4월은 내 생일: 4월 16일 30대 여성(1) 고가품목(10) 구매수량 400% 증대
mydf.loc[(mydf.age>=30)&(mydf.age<40)&(mydf.gender==1)
         &((mydf.index=='2020-04-16')|(mydf.index=='2021-04-16')),'quantity']*=4
mydf.quantity = mydf.quantity.astype(int)
mydf['총 구매액'] = mydf.price*mydf.quantity # 총 구매액 갱신
mydf

,cusno,gender,age,region,timedata,Year,Month,Day,time,price,product,quantity,총 구매액
timedata,,,,,,,,,,,,,
2020-01-01,578,0,64,3,2020-01-01,2020,1,WED,10,26500,20,2,53000
2020-01-01,862,0,4,4,2020-01-01,2020,1,WED,20,1600,30,64,102400
2020-01-01,97,0,27,4,2020-01-01,2020,1,WED,15,7100,30,79,560900
2020-01-01,525,0,77,4,2020-01-01,2020,1,WED,16,13700,30,23,315100
2020-01-01,149,1,34,3,2020-01-01,2020,1,WED,1,42200,10,1,42200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31,826,1,72,8,2021-12-31,2021,12,FRI,4,47900,10,28,1341200
2021-12-31,883,1,30,4,2021-12-31,2021,12,FRI,8,16300,30,42,684600
2021-12-31,283,0,29,9,2021-12-31,2021,12,FRI,13,39900,10,2,79800


In [12]:
data = mydf.loc[:,['gender','age','Year','Month','Day',
               'time','price','product','quantity','총 구매액']]
gr = data.groupby(['Year','Month'])
gr.mean().head()

gender        age       time         price   quantity  \
Year Month                                                            
2020 1      0.490610  40.915493  12.795775  24889.436620  51.884977   
     2      0.464467  41.121827  12.159898  24783.248731  48.644670   
     3      0.484848  40.348485  12.809524  26177.705628  51.510823   
     4      0.462830  41.019185  13.038369  23526.139089  51.491607   
     5      0.474940  41.322196  13.100239  25920.286396  51.028640   

                   총 구매액  
Year Month                
2020 1      1.286072e+06  
     2      1.182310e+06  
     3      1.366910e+06  
     4      1.245941e+06  
     5      1.332378e+06

In [132]:
# 연령대 범주화
def agecat(age):
    cat = ''
    if age<10: cat='아동'
    elif age<20: cat='10대'
    elif age<30: cat='20대'
    elif age<40: cat='30대'
    elif age<50: cat='40대'
    elif age<60: cat='50대'
    elif age<70: cat='60대'
    else: cat='노년층'
    return cat
data['age'] = data.age.apply(lambda x:agecat(x))

In [133]:
gr = data.groupby(['Year','age'])
gr.mean().round(3)
# 전체적으로 구매액이 30~50대에서 높게 나온다.

gender  Month    time      price  quantity        총 구매액
Year age                                                         
2020 10대   0.513  6.550  12.689  24867.692    35.566   795148.547
     20대   0.605  6.377  12.369  25716.279    67.849  1715592.857
     30대   0.562  6.565  12.212  24819.635   133.645  3298585.084
     40대   0.431  6.635  12.321  25636.834   127.914  3135247.819
     50대   0.525  6.637  12.522  24030.847   132.673  3105046.102
     60대   0.473  6.408  12.626  26064.545    70.947  1716757.727
     노년층   0.508  6.564  12.570  25944.756    71.787  1827458.493
     아동    0.479  6.489  12.516  24830.674     2.746    66937.411
2021 10대   0.524  6.524  12.956  24944.444    38.517   866926.431
     20대   0.566  6.538  12.246  25529.816    71.486  1794513.735
     30대   0.587  6.661  12.893  25501.227   135.847  3515437.117
     40대   0.415  6.603  12.413  25716.382   128.474  3296355.556
     50대   0.528  6.736  12.810  25598.831   139.783  3353975.125
     60대   0.472  6.558  12.524  24795.793    74.385  1833784.951
     노년층   0.520  6.509  12.635  24691.311    74.595  1802295.584
     아동    0.503  6.725  12.668  25236.426     2.773    68150.515

In [134]:
# 연도별 나이 - 구매시기 상관관계
pv1 = pd.pivot_table(data,
                     index = ['Year','age'], # 행 위치에 들어갈 열
                     columns = 'Month', # 열 위치에 들어갈 열
                     values = '총 구매액', # 데이터로 사용할 열
                     aggfunc = 'mean' # 데이터 집계 함수
                     ).round(3)
pv1
# 10대의 3월 소비금액이 커졌다. 이건 내가 신학기 변인을 넣었기 때문
# 40대, 50대의 2월 소비금액이 눈에 띄게 커졌다. 이거 성별로 나눠서 봐야 할 듯
# 9월은 명절 보정 + 재난지원금 보정 때문에 구매금액이 폭발적으로 늘어남

Month              1            2            3            4            5   \
Year age                                                                    
2020 10대   773913.043   955725.000  1109246.429   582988.889   815987.037   
     20대  1389517.647  1567489.831  1528022.807  1596092.308  1736466.667   
     30대  2820278.261  6009059.574  3000378.182  2469355.556  2581075.410   
     40대  2096574.419  3249985.714  2424854.348  2548832.727  2757600.000   
     50대  3526925.000  4601604.167  1924743.137  2375885.185  2718491.667   
     60대  1174816.364  3295075.000  1870157.447  1638449.057  1549386.154   
     노년층  1512504.615  3756780.000  1419128.571  1622191.837  1435023.333   
     아동     56225.455    89700.000    83619.048    75412.000    88764.583   
2021 10대   657434.783   733402.041  1014804.167   927108.333   764950.000   
     20대  1913559.649  1735768.966  2264950.000  1461501.818  1214148.837   
     30대  3178209.434  7616104.000  2709402.128  2433684.746  2213178.689   
     40대  2616872.222  5813181.132  2660250.746  2002138.298  2328656.667   
     50대  2939721.569  7093617.143  2788882.000  2200686.047  2509322.000   
     60대  1477517.391  4193567.568  1383098.246   987392.453  1125759.016   
     노년층  1674410.769  2924863.830  1818812.281  1589321.875  1335472.973   
     아동     77623.810    74969.565    76018.033    42200.000    59381.395   

Month              6            7            8            9            10  \
Year age                                                                    
2020 10대   563454.167   909331.148   601600.000   714219.048   810056.364   
     20대  1643942.000  2432507.547  2441359.016  1531977.500  1179472.973   
     30대  2735068.852  4342560.345  4245529.545  4481781.481  2534082.540   
     40대  2608609.434  4230595.556  4419350.909  5186810.256  2918328.333   
     50대  2791778.846  3495050.000  2863081.081  5741733.333  2471196.552   
     60대  1192542.373  1460333.333  1242562.903  3103166.038  1320353.333   
     노년층  1794401.923  1552236.364  1467640.351  3389202.128  1616695.082   
     아동     51140.000    60748.936    51640.741    69556.863    56076.316   
2021 10대  1167252.000   821232.787   799460.714   948057.500   713318.519   
     20대  1211802.174  2346286.047  2527849.153  1862190.476  1459537.037   
     30대  2219578.000  4718294.118  3399946.269  6519813.208  2587706.897   
     40대  3620798.113  3836729.730  3633363.158  4843710.714  2405265.000   
     50대  3005425.926  3456452.632  2648264.407  6095372.308  2859830.000   
     60대  1568910.526  1977420.370  1581849.057  4240636.957  1595125.862   
     노년층  1793604.255  1575161.538  1232225.424  2973969.643  1625700.000   
     아동     65515.217    56844.068    72859.091    67742.857    56254.348   

Month              11           12  
Year age                            
2020 10대   809278.723   783006.000  
     20대  1321426.531  1851318.182  
     30대  2061485.714  3014085.714  
     40대  2351877.778  3400658.824  
     50대  1744733.333  2823724.490  
     60대  1120584.483  1822686.364  
     노년층  1200300.000  2041864.151  
     아동     57595.918    76188.095  
2021 10대   924358.333   974084.783  
     20대  1095192.308  2016422.449  
     30대  2175019.048  3596857.895  
     40대  2288470.968  3596683.051  
     50대  2252039.623  2801066.667  
     60대  1694893.333  1302121.569  
     노년층  1586445.614  1903550.000  
     아동     59072.549    95362.069

In [135]:
# 월별 남녀 소비금액 비교

data1 = data.loc[:,['gender','age','Month',
               'price','product','quantity','총 구매액']]
gr = data1.groupby(['Month','gender'])
for i,j in gr:
    display(i,j.head())
    print(f"{j['총 구매액'].sum()}원")
# 위에서 의문을 품었던 2, 9월 소비금액 증대는
# 여성 고객이 남성 고객에 비해 거의 3배쯤 소비를 더 많이 해서 나온 결과임(실제 증가한 값도 300%)
# 휴가 시즌, 홀리데이 시즌엔 남녀 구분 없이 구매금액이 늘어남 - 시즈널 이펙트

(1, 0)

,gender,age,Month,price,product,quantity,총 구매액
timedata,,,,,,,
2020-01-01,0,30대,1,40800,10,83,3386400
2020-01-01,0,30대,1,36200,10,34,1230800
2020-01-01,0,20대,1,8100,30,38,307800
2020-01-02,0,아동,1,28900,20,10,289000
2020-01-02,0,아동,1,15500,30,1,15500


624791200원


(1, 1)

,gender,age,Month,price,product,quantity,총 구매액
timedata,,,,,,,
2020-01-01,1,60대,1,10800,30,8,86400
2020-01-01,1,40대,1,1200,30,166,199200
2020-01-01,1,30대,1,31600,20,175,5530000
2020-01-01,1,30대,1,23900,20,260,6214000
2020-01-01,1,30대,1,22400,20,244,5465600


...

,gender,age,Month,price,product,quantity,총 구매액
timedata,,,,,,,
2020-12-01,0,10대,12,4800,30,70,336000
2020-12-01,0,60대,12,32800,20,207,6789600
2020-12-01,0,노년층,12,17300,30,38,657400
2020-12-01,0,40대,12,45600,10,33,1504800
2020-12-01,0,10대,12,10100,30,25,252500


824771900원


(12, 1)

,gender,age,Month,price,product,quantity,총 구매액
timedata,,,,,,,
2020-12-01,1,20대,12,34800,10,63,2192400
2020-12-01,1,20대,12,49400,10,97,4791800
2020-12-01,1,아동,12,38900,10,3,116700
2020-12-01,1,20대,12,1700,30,8,13600
2020-12-01,1,10대,12,3500,30,65,227500


882054200원


In [154]:
df416 = data[(data.gender==1)&(data.Month==4)&(data.age=='30대')]
df416.drop(['Year','time','gender','Month'],axis=1,inplace=True)
df416['일자'] = df416.index
df416['일자'] = df416.일자.dt.day
gr416 = df416.groupby(['일자'])
gr416.mean().round(3).head(50)
# 4월 16일 가중치를 열심히 넣었지만 눈에 띄게 늘어나진 않았다. 그래도 다른 날보다 높은 구매액이긴 함

,price,quantity,총 구매액
일자,,,
1,25800.000,59.250,1.416450e+06
2,36700.000,37.000,1.357900e+06
3,34300.000,47.500,1.873200e+06
4,25400.000,106.000,3.108200e+06
5,48650.000,56.000,2.745650e+06
6,35833.333,80.667,2.850133e+06
8,8866.667,90.667,9.126000e+05
9,32200.000,48.667,1.478333e+06
10,28000.000,42.500,4.512500e+05


In [112]:
# 연도별 시간대-요일간 상관관계
# 왜 니맘대로 요일을 정렬하니 ㅠㅠ
pv1 = pd.pivot_table(data,
                     index = 'time', # 행 위치에 들어갈 열
                     columns = 'Day', # 열 위치에 들어갈 열
                     values = ['총 구매액','quantity'], # 데이터로 사용할 열
                     aggfunc = 'mean' # 데이터 집계 함수
                     ).round(3)
pv1

# 월요일은 전체적으로 침체된 소비 패턴
# 시간대에선 영업시간 보정을 넣었기 때문에 그때부터 본격적인 구매량 증가
# 주말은 랜덤임을 감안해도 가중치가 있어서 전체적으로 높은 소비 금액을 보이고 있다


quantity                                                       \
Day       FRI     MON      SAT      SUN      THU      TUE      WED   
time                                                                 
1      68.015  26.688   56.517   61.123   59.708   48.464   43.509   
2      66.095  39.389   63.677   68.095   56.818   56.770   63.591   
3      64.246  33.260   59.431   90.804   77.698   38.727   58.558   
4      59.400  34.046   74.879   38.842   60.040   58.587   50.085   
5      74.691  24.848   54.447   68.094   76.033   58.234   47.724   
6      58.517  25.766   55.291   59.000   57.141   51.582   58.185   
7      57.619  34.148   45.770   49.966   73.918   69.149   63.338   
8      52.281  25.000   87.061   51.155   55.186   56.578   44.379   
9      68.467  33.375   72.519   50.709   46.552   50.017   59.382   
10    176.179  66.482  133.429  108.703  112.426  126.180  107.321   
11    142.525  63.915   89.548  114.548  107.544  137.250  129.764   
12    123.750  54.841  115.444  132.393  100.684  115.429  112.736   
13    128.631  70.852   99.170  148.281  108.554  149.451  154.375   
14    117.614  47.473  120.558  124.348   83.600  128.390  147.836   
15     97.302  42.889   99.732  108.596  106.344  131.275  121.492   
16    105.254  47.156  129.200  162.656   88.340  116.424  134.491   
17    111.508  49.597  152.738  129.113   97.622  115.613   95.065   
18    120.698  46.967  132.686  219.259  113.034  107.841  133.200   
19    136.508  56.984  131.758  174.500  104.889  103.308  103.982   
20    126.643  51.121  117.517  124.766   99.986  113.581  113.310   
21    102.343  38.803  107.463   87.952   89.600   84.100   93.018   
22    109.394  42.400  156.562  102.472   82.061   89.146   76.600   
23     39.333  17.683   50.960   58.571   54.231   42.762   49.323   
24     54.823  28.071   43.686   36.959   30.642   34.911   38.909   

            총 구매액                                                      \
Day           FRI          MON          SAT          SUN          THU   
time                                                                    
1     1625907.692   669281.250  1407871.667  1327093.846  1267798.462   
2     1511907.143  1007404.167  1647527.419  1942571.429  1299254.545   
3     1776739.130   954424.000  1540861.538  2012241.176  1962324.528   
4     1310341.538   915787.692  1789307.576   983638.596  1757944.000   
5     1974556.364   581368.182  1537419.149  1831794.340  1872660.656   
6     1514755.000   610764.062  1395810.909  1365109.756  1532178.125   
7     1500204.762   915022.951  1043506.557  1245850.847  1656737.705   
8     1371979.688   624075.000  1943759.091  1276055.172  1505962.712   
9     1545466.667   770225.000  1534961.538  1267603.636  1197760.345   
10    3995150.746  1541508.929  3378103.175  2514334.375  2559738.889   
11    3685332.787  1720644.681  1936393.548  2952916.129  2501905.882   
12    3385967.857  1274803.175  3062587.302  3240571.429  2447877.193   
13    2980956.923  1604350.000  2642844.681  3919359.375  2971769.231   
14    2570138.596  1285036.364  2940936.538  3248262.121  2084836.364   
15    2217860.317  1148636.111  2368071.429  2434674.468  2824880.328   
16    2789738.095  1165881.250  3071283.077  3681778.125  2042518.868   
17    2999791.803  1213285.075  3887429.231  3218471.698  2509973.333   
18    2787304.762  1377065.574  3229472.549  5954790.741  2726229.310   
19    3488608.197  1531479.688  3325401.613  4271700.000  2456519.048   
20    2580117.857  1319725.862  3378568.966  3101925.000  2584617.391   
21    2835561.194  1014201.639  2802223.881  2206196.825  2144085.000   
22    2882457.576   993874.545  3937106.250  2687298.113  2216538.776   
23    1010787.037   435615.873  1230840.000  1309730.159  1317772.308   
24    1524343.548   667896.429   925568.571   889578.378   687260.377   

                                
Day           TUE          WED  
time                            
1     1312323.214   970433.962  
2     1329

In [89]:
data.corr() # 변수 간의 상관관계는 크게 없는 듯

,gender,Year,Month,time,price,quantity,총 구매액
gender,1.000000,-0.001025,-0.008333,-0.001251,0.003135,0.090408,0.086691
Year,-0.001025,1.000000,0.008153,0.002106,-0.008731,-0.021555,-0.029000
Month,-0.008333,0.008153,1.000000,0.008392,-0.004780,-0.001188,-0.015870
time,-0.001251,0.002106,0.008392,1.000000,-0.002548,-0.039413,-0.037194
price,0.003135,-0.008731,-0.004780,-0.002548,1.000000,-0.018179,0.246496
quantity,0.090408,-0.021555,-0.001188,-0.039413,-0.018179,1.000000,0.892607
총 구매액,0.086691,-0.029000,-0.015870,-0.037194,0.246496,0.892607,1.000000
